In [1]:
import csv
import pandas
import numpy
import random
from CWHashingFamily import CWHashingFamily

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [11]:

ID = 0
TEXT = 9
R = 4
B = 8

#trainRDD = sc.textFile("train_sin_repeticiones.csv").map(lambda line: line.split(","))
trainRDD = sc.textFile("train_sin_repeticiones.csv").mapPartitions(lambda x: csv.reader(x))

#sql = SQLContext(sc)
#trainRDD = sc.read.option("header", "true").csv("train_sin_repeticiones.csv")

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

print trainRDD.take(2)

trainRDD = trainRDD.map(lambda line: (line[ID], shingles(line[TEXT],SHINGLES_LEN)))


trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] )).take(5)

#print trainRDD


[['492346', 'B001VNP0Y6', 'AEZRVY3CV52UZ', 'Corey Wright Democratus', '3', '3', '5', '1320537600', 'just what i needed', "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."], ['343811', 'B008J1HO4C', 'A18VZYSLOUO060', 'D. Simpson Frugal SOB', '2', '2', '5', '1204329600', 'most excellent oatmeal', "mccann's steel cut oatmeal is the perfect breakfast for people in a hurry.  all you do is put 1/2 cup of oats in a little crockpot with 2 cups of boiling water before you go to bed, then an hour before i get up the crockpot starts on high on a timer so when i get out of the shower the oatmeal is ready to go.  just mix in some blueberries, raisins, craisins, dates, nuts or whatever you want and eat."]]


In [44]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)

for i in xrange(len(trainRDD)):
    record = Record((trainRDD[i][0], trainRDD[i][1]))
    lsh.add(record)


Adding record:492346 to LSH at POS:26
table 0 :['492346']
Adding record:492346 to LSH at POS:49
table 1 :['492346']
Adding record:492346 to LSH at POS:10
table 2 :['492346']
Adding record:492346 to LSH at POS:60
table 3 :['492346']
Adding record:492346 to LSH at POS:16
table 4 :['492346']
Adding record:492346 to LSH at POS:69
table 5 :['492346']
Adding record:492346 to LSH at POS:0
table 6 :['492346']
Adding record:492346 to LSH at POS:52
table 7 :['492346']
Adding record:343811 to LSH at POS:20
table 0 :['343811']
Adding record:343811 to LSH at POS:10
table 1 :['343811']
Adding record:343811 to LSH at POS:0
table 2 :['343811']
Adding record:343811 to LSH at POS:8
table 3 :['343811']
Adding record:343811 to LSH at POS:24
table 4 :['343811']
Adding record:343811 to LSH at POS:36
table 5 :['343811']
Adding record:343811 to LSH at POS:96
table 6 :['343811']
Adding record:343811 to LSH at POS:8
table 7 :['343811']
Adding record:347257 to LSH at POS:78
table 0 :['347257']
Adding record:3472

In [50]:
shinglesText = ['"i g', 'i go', ' got', 'got ', 'ot t', 't th', ' the', 'them', 'hem ', 'em i', 'm in', ' in ', 'in a', 'n a ', ' a v', 'a ve', ' ver', 'very', 'ery ', 'ry t', 'y ti', ' tim', 'time', 'imel', 'mely', 'ely ', 'ly m', 'y ma', ' man', 'mann', 'anne', 'nner', 'ner ', 'er a', 'r an', ' and', 'and ', 'nd t', 'd th', ' the', 'they', "hey'", "ey'r", "y're", "'re ", 're a', 'e al', ' all', 'all ', 'll v', 'l ve', ' ver', 'very', 'ery ', 'ry l', 'y la', ' lar', 'larg', 'arge', 'rge ', 'ge a', 'e an', ' and', 'and ', 'nd g', 'd gr', ' gre', 'gree', 'reen', 'een.', 'en. ', 'n. t', '. th', ' the', 'ther', 'here', 'ere ', 're s', 'e se', ' see', 'seem', 'eems', 'ems ', 'ms t', 's to', ' to ', 'to b', 'o be', ' be ', 'be a', 'e an', ' an ', 'an i', 'n in', ' ino', 'inor', 'nord', 'ordi', 'rdin', 'dina', 'inat', 'nate', 'ate ', 'te a', 'e am', ' amo', 'amou', 'moun', 'ount', 'unt ', 'nt o', 't of', ' of ', 'of t', 'f th', ' the', 'the ', 'he s', 'e se', ' see', 'seed', 'eeds', 'eds ', 'ds e', 's e ', ' e o', 'e ou', ' out', 'out ', 'ut a', 't a ', ' a w', 'a we', ' wei', 'weir', 'eird', 'ird ', 'rd b', 'd be', ' bei', 'beig', 'eige', 'ige ', 'ge c', 'e co', ' col']
test = (492346,shinglesText)
print test
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
testRecord = Record(test)
print test



(492346, ['"i g', 'i go', ' got', 'got ', 'ot t', 't th', ' the', 'them', 'hem ', 'em i', 'm in', ' in ', 'in a', 'n a ', ' a v', 'a ve', ' ver', 'very', 'ery ', 'ry t', 'y ti', ' tim', 'time', 'imel', 'mely', 'ely ', 'ly m', 'y ma', ' man', 'mann', 'anne', 'nner', 'ner ', 'er a', 'r an', ' and', 'and ', 'nd t', 'd th', ' the', 'they', "hey'", "ey'r", "y're", "'re ", 're a', 'e al', ' all', 'all ', 'll v', 'l ve', ' ver', 'very', 'ery ', 'ry l', 'y la', ' lar', 'larg', 'arge', 'rge ', 'ge a', 'e an', ' and', 'and ', 'nd g', 'd gr', ' gre', 'gree', 'reen', 'een.', 'en. ', 'n. t', '. th', ' the', 'ther', 'here', 'ere ', 're s', 'e se', ' see', 'seem', 'eems', 'ems ', 'ms t', 's to', ' to ', 'to b', 'o be', ' be ', 'be a', 'e an', ' an ', 'an i', 'n in', ' ino', 'inor', 'nord', 'ordi', 'rdin', 'dina', 'inat', 'nate', 'ate ', 'te a', 'e am', ' amo', 'amou', 'moun', 'ount', 'unt ', 'nt o', 't of', ' of ', 'of t', 'f th', ' the', 'the ', 'he s', 'e se', ' see', 'seed', 'eeds', 'eds ', 'ds e'

In [51]:
print lsh.getAllSimilarRecords(testRecord)

obtained pos:26 for record (492346,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 0
Getting records from POS:26 from: ['492346', '225529']
obtained pos:49 for record (492346,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 1
Getting records from POS:49 from: ['492346']
obtained pos:10 for record (492346,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 2
Getting records from POS:10 from: ['492346']
obtained pos:60 for record (492346,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 3
Getting records from POS:60 from: ['492346']
obtained pos:16 for record (492346,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 4
Getting records from POS:16 from: ['492346']
obtained pos:69 for record (492346,[1, 0, 1, 0, 1,

4
3
4
3
1
10
10
1
